In [ ]:
import Test.QuickCheck
data FixMePls a = 
    FixMe
    | Pls a
    deriving (Eq,Show)
instance Functor FixMePls where
    fmap _ FixMe =  FixMe
    fmap f (Pls a) = Pls (f a)


In [ ]:
data WhoCares a = 
    ItDoesnt
    | Matter a
    | WhatThisIsCalled
    deriving (Eq,Show)
instance Functor WhoCares where
    fmap f (Matter a) = Matter (f a)
    fmap _ ItDoesnt = ItDoesnt
    fmap _ WhatThisIsCalled = WhatThisIsCalled

In [ ]:
data CountingBad a = 
    Heisenberg Int a
    deriving (Eq,Show)
instance Functor CountingBad where
    fmap f (Heisenberg n x) = Heisenberg (n) (f x)
oneWhoKnocks = Heisenberg 0 "Uncle"
fmap (++ " Jesse") oneWhoKnocks

In [ ]:
:t const
replaceWithP = const 'p'
-- replaceWithP 10000
-- replaceWithP "woohoo"
-- replaceWithP (Just 10)

fmap replaceWithP (Just 10)
fmap replaceWithP Nothing

fmap  replaceWithP [1,2,3,4,5]
fmap replaceWithP "Ave"

fmap (+1) []
:t fmap replaceWithP [] -- why type changed?

-- data (,) a b = (,) a b
fmap replaceWithP (10,20)
fmap replaceWithP (10,"woo")


In [ ]:
tossEmOne = fmap (+1) negate
tossEmOne 10
tossEmOne (-10)

In [ ]:
lms = [Just "Ave",Nothing,Just "woohoo"]
fmap replaceWithP lms
(fmap.fmap) replaceWithP lms
(fmap.fmap.fmap) replaceWithP lms

In [ ]:
:t fmap.fmap

In [ ]:
a = fmap (+1) $ read "[1]" :: [Int]
b = (fmap.fmap) (++ "lol") (Just ["Hi,", "Hello"])
c =fmap (*2) (\x -> x - 2)
d = fmap ((return '1' ++) .show) (\x -> [x,1..3])
d 0

In [ ]:
e :: IO Integer  
e = let ioi = readIO "1" :: IO Integer  
        changed = fmap (read.  ("123"++) . show) ioi  
    in fmap (*3) changed
:t e

In [ ]:
read ("123"++"1" ) :: Integer

In [ ]:
functorIdentity :: (Functor f,Eq (f a)) => f a -> Bool
functorIdentity f = fmap id f == f
functorCompose :: (Eq (f c),Functor f) => (a->b)->(b->c)->f a ->Bool
functorCompose f g x = (fmap g (fmap f x)) == (fmap (g . f ) x )

quickCheck $ \x -> functorIdentity (x :: [Int])

li x = functorCompose (+1)(*2) (x :: [Int])
quickCheck li

In [ ]:
import Test.QuickCheck
import Test.QuickCheck.Function

functorCompose' :: (Eq (f c), Functor f) => f a
                                         -> Fun a b
                                         -> Fun b c
                                         -> Bool
functorCompose' x (Fun _ f) (Fun _ g) = (fmap (g . f) x) == (fmap g .fmap  f $ x)
type IntToInt = Fun Int Int
type IntFC = [Int]-> IntToInt -> IntToInt -> Bool
quickCheck (functorCompose' :: IntFC)

In [ ]:
newtype Identity a = Identity a deriving (Eq,Show)
instance Functor Identity where
	fmap f (Identity a) = Identity (f a)
instance Arbitrary a => Arbitrary (Identity a) where
    arbitrary = Identity <$> arbitrary
type IntToInt = Fun Int Int
type IntIden = Identity Int -> IntToInt->IntToInt->Bool
quickCheck $ \x -> functorIdentity (x:: Identity Int)
quickCheck (functorCompose' :: IntIden)

In [ ]:
data Pair a = Pair a a deriving (Eq,Show)
instance Functor Pair where
    fmap f (Pair a a') = Pair (f a) (f a')
instance Arbitrary a =>Arbitrary (Pair a) where
    arbitrary = Pair <$> arbitrary <*> arbitrary
type IntPair = Pair Int -> IntToInt -> IntToInt -> Bool
quickCheck (functorIdentity :: Pair Int -> Bool)
quickCheck (functorCompose' :: IntPair)

In [ ]:
data Two a b = Two a b deriving (Eq,Show)
instance Functor (Two a) where
    fmap f (Two a b) =Two a (f b)
instance (Arbitrary a,Arbitrary b) =>Arbitrary (Two a b) where
    arbitrary = Two <$> arbitrary <*> arbitrary
type IntTwo = Two Int Int -> IntToInt -> IntToInt -> Bool
quickCheck (functorIdentity :: Two Int Int -> Bool)
quickCheck (functorCompose' :: IntTwo)

In [ ]:
data Four a b c d = Four a b c d deriving (Eq,Show)
instance Functor (Four a b c) where
    fmap f (Four a b c d) = Four a b c (f d)
instance (Arbitrary a,Arbitrary b,Arbitrary c,Arbitrary d)=>Arbitrary (Four a b c d) where
    arbitrary = Four <$> arbitrary <*> arbitrary <*> arbitrary <*> arbitrary
type IntFour = Four Int Int Int Int -> IntToInt->IntToInt->Bool

quickCheck (functorIdentity :: Four Int Int Int Int -> Bool)
quickCheck (functorCompose' :: IntFour)

In [ ]:
data Four' a b  = Four' a a a b deriving (Eq,Show)
instance Functor (Four' a ) where
    fmap f (Four' a1 a2 a3 b) = Four' a1 a2 a3 (f b)
instance (Arbitrary a,Arbitrary b)=>Arbitrary (Four' a b) where
    arbitrary = Four' <$> arbitrary <*> arbitrary <*> arbitrary <*> arbitrary
type IntFour' = Four' Int Int  -> IntToInt->IntToInt->Bool

quickCheck (functorIdentity :: Four' Int Int -> Bool)
quickCheck (functorCompose' :: IntFour')

In [ ]:
data Trivial = Trivial deriving (Eq,Show)


In [ ]:
liftedInc :: (Functor f ,Num b)=> f b -> f b
liftedInc = fmap (+1)

liftedShow :: (Functor f, Show a)=>f a -> f String
liftedShow = fmap show


In [ ]:
liftedInc (Just 2)
liftedShow Nothing
liftedInc [1..6]
liftedShow [1..6]

In [ ]:
data Possibly a = 
    LolNope
    | Yeppers a
    deriving (Eq,Show)
instance Functor Possibly where
    fmap _ LolNope = LolNope
    fmap f (Yeppers x) = Yeppers ( f x)

fmap (+1) (Yeppers 1)
fmap (+1) Nothing

In [ ]:
newtype Constant a b = Constant {getConstant :: a}
    deriving (Eq,Show)
instance Functor (Constant m ) where
    fmap _ (Constant v) = Constant v
const 2 (getConstant (Constant 3))
fmap (const 2) (Constant 3)

In [ ]:
import Data.Monoid
data Wrap f a = 
    Wrap (f a)
    deriving (Eq ,Show)
instance Functor f=> Functor(Wrap f) where
    fmap f (Wrap fa) = Wrap (fmap f fa)
fmap (+1) (Wrap [1,2,3])
fmap (+1) (Wrap (Sum 1))

In [ ]:
getInt :: IO Int
getInt = fmap read getLine
-- getInt = 10 :: Int
-- const () getInt
-- fmap (const ()) getInt
return 1 :: IO Int

In [ ]:
type Nat f g = forall a . f a -> g a

In [ ]:
maybeToList :: Nat Maybe []
maybeToList Nothing = []
maybeToList (Just a) = [a]

-- maybeToList (Just 1)

In [ ]:
-- degenerateMtl :: Num a => Nat Maybe []  
-- degenerateMtl Nothing = []  
-- degenerateMtl (Just a) = [a+1]

## Chapter Exercises

In [ ]:
newtype Mu f = InF {outF :: f (Mu f)}


-- newtype Mu List = InF {outF :: List (Mu List)}
-- instance Functor f => Functor Mu where
--     fmap f (InF x) = InF (fmap (fmap f) x)
instance Functor f => Functor (Mu f) where
    fmap f (InF x) = InF (fmap (fmap f) x)

In [ ]:
import GHC.Arr
data D = D (Array Word Word) Int Int

In [ ]:
newtype Flip f a b = Flip ( f b a) deriving (Eq,Show)
newtype K a b = K a
instance Functor (Flip K a) where
    fmap f (Flip (K a)) = Flip(K (f a))

In [ ]:
data EvilGoateeConst a b = GoatyConst b
instance Functor (EvilGoateeConst a) where
    fmap f (GoatyConst b) = GoatyConst (f b)
    

In [ ]:
data Parappa f g a = Dawrappa (f a) (g a)
instance (Functor f, Functor g) => Functor (Parappa f g) where
    fmap f (Dawrappa fa ga) = Dawrappa (fmap f fa) (fmap f ga)

In [ ]:
data IgnoreOne f g a b= IgnoringSomething (f a) (g b)
instance (Functor g)=>Functor (IgnoreOne f g a) where
    fmap f (IgnoringSomething fa gb) = IgnoringSomething fa (fmap f gb)

data Notorious g o a t = Notorious (g o) (g a) (g t)
instance (Functor g)=>Functor (Notorious g o a) where
    fmap f (Notorious go ga gt) = Notorious go ga (fmap f gt)

In [ ]:
data List a = Nil | Cons a (List a) deriving (Eq,Show)
instance Functor List where
    fmap _ Nil = Nil
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)


In [156]:
data GoatLord a =  NoGoat  
    | OneGoat a  
    | MoreGoats (GoatLord a) (GoatLord a) (GoatLord a)
    deriving (Eq,Show)
instance Functor GoatLord where
    fmap _ NoGoat = NoGoat
    fmap f (OneGoat x) = OneGoat (f x)
    fmap f (MoreGoats l m r )= MoreGoats (fmap f l) (fmap f m) (fmap f r)

MoreGoats (OneGoat 1) (OneGoat 2) (OneGoat 3)

MoreGoats (OneGoat 1) (OneGoat 2) (OneGoat 3)

In [157]:
data TalkToMe a = 
        Halt
        | Print String a
        | Read (String -> a)
instance Functor TalkToMe where
    fmap _ Halt = Halt
    fmap f (Print s a)=Print s (f a)
    fmap f (Read g) = Read (f.g)